# 🫁 Qwen2.5 Chest X-ray Abnormality Grounding

Locate abnormality areas in chest X-rays using Qwen2.5 and evaluate with mAP.

## 1. Setup & Imports

In [29]:
import os, json, base64
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
%matplotlib inline

def encode_image_to_data_uri(path: str) -> str:
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode('utf-8')
    return f"data:image/png;base64,{b64}"

DATASET_DIR = "VLM-Seminar25-Dataset/chest_xrays"
IMAGES_DIR = os.path.join(DATASET_DIR, "images")
ANNOT_PATH = os.path.join(DATASET_DIR, "annotations_len_50.json")
RESULTS_DIR = "../results/chest_xrays/grounding"

os.makedirs(RESULTS_DIR, exist_ok=True)

with open(ANNOT_PATH, "r") as f:
    annotations = json.load(f)
image_ids = list(annotations.keys())
print(f"Number of images in the dataset: {len(image_ids)}")
print(f"Number of images in the dataset: {len(list(set(image_ids)))}")

load_dotenv(dotenv_path="../config/user.env")
api_key = os.environ.get("NEBIUS_API_KEY")
client = OpenAI(base_url="https://api.studio.nebius.com/v1/", api_key=api_key)

Number of images in the dataset: 50
Number of images in the dataset: 50


In [23]:
do_new_inference = True

## 2. Model Inference
Only done for unhealthy xrays!

In [ ]:
grounding_results = []
if do_new_inference:
    for img_id in tqdm(image_ids):
        print()
        ann = annotations[img_id]
        if ann["status"] == "healthy" or not ann.get("global_disease"):
            continue
        disease = ann["global_disease"][0]
        img_path = os.path.join(IMAGES_DIR, img_id + ".png")
        data_uri = encode_image_to_data_uri(img_path)
        prompt = f"Please locate {disease} and output bounding boxes as floats [x1, y1, x2, y2]. Output nothing else."
        completion = client.chat.completions.create(
            model="Qwen/Qwen2.5-VL-72B-Instruct",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {"type": "image_url", "image_url": {"url": data_uri}},
                    ],
                }
            ],
        )
        pred = completion.choices[0].message.content.strip()
        print(pred)
        grounding_results.append({"id": img_id, "disease": disease, "prediction": pred})

Interesting prompting note: Without  "If the disease is not present, output '[]'." it outputs coords, otherwise often not!

## 3. Save Model Predictions

In [26]:
with open(os.path.join(RESULTS_DIR, "qwen2.5_grounding_results.json"), "w") as f:
    json.dump(grounding_results, f, indent=2)
print("Saved grounding results.")

Saved grounding results.


Load results

In [40]:
with open(os.path.join(RESULTS_DIR, "qwen2.5_grounding_results.json"), "r") as f:
    grounding_results = json.load(f)
print(f"Number of grounding results: {len(grounding_results)}")

Number of grounding results: 12


## 4. Evaluation & Metrics

In [42]:
sys.path.append("eval_scripts")
from calculate_map import compute_map_supervision, draw_boxes

